<a href="https://colab.research.google.com/github/podilapujagadeesh/FMML-2023_PROJECTS_and_SUBMISSIONS/blob/main/Mod3_lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FOUNDATIONS OF MODERN MACHINE LEARNING, IIIT Hyderabad**
### MODULE: CLASSIFICATION-1
### LAB-3 : Using KNN for Text Classification
#### Module Coordinator: Jashn Arora


---

## **Section 1: Understanding NLP tools**

In this lab we will be using KNN on a real world NLP application i.e. is text classification. But first look at some NLP techniques for text classification and tools that we use when we want to use python for NLP.

## Section 1.2: Data Cleaning and Preprocessing step

Raw text must be processed and converted into a form so that it is suitable to use with various machine-learning algorithms.  
In case of text, there are lots of things that need to be taken into account.  


1.   Removing numbers from the text
2.   Handling capitalization and punctuation.
3.   Stemming and Lemmatizing text.  

And most importantly, one can't just use words or images directly in algorithms; they need to be converted into vectors- a form that algorithms can understand.



### **NLTK**
NLTK (or Natural Language Tool Kit) is a commonly used library for processing text. We will use this tool in this lab. Lets first install it.


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
import re
import numpy
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from bs4 import BeautifulSoup

def cleanText(text, lemmatize, stemmer):
    """Method for cleaning text from train and test data. Removes numbers, punctuation, and capitalization. Stems or lemmatizes text."""

    if isinstance(text, float):
        text = str(text)
    if isinstance(text, numpy.int64):
        text = str(text)
    try:
        text = text.decode()
    except AttributeError:
        pass

    soup = BeautifulSoup(text, "lxml")
    text = soup.get_text()
    text = re.sub(r"[^A-Za-z]", " ", text)
    text = text.lower()


    if lemmatize:
        wordnet_lemmatizer = WordNetLemmatizer()

        def get_tag(tag):
            if tag.startswith('J'):
                return wordnet.ADJ
            elif tag.startswith('V'):
                return wordnet.VERB
            elif tag.startswith('N'):
                return wordnet.NOUN
            elif tag.startswith('R'):
                return wordnet.ADV
            else:
                return ''

        text_result = []
        tokens = word_tokenize(text)  # Generate list of tokens
        tagged = pos_tag(tokens)
        for t in tagged:
            try:
                text_result.append(wordnet_lemmatizer.lemmatize(t[0], get_tag(t[1][:2])))
            except:
                text_result.append(wordnet_lemmatizer.lemmatize(t[0]))
        return text_result

    if stemmer:
        text_result = []
        tokens = word_tokenize(text)
        snowball_stemmer = SnowballStemmer('english')
        for t in tokens:
            text_result.append(snowball_stemmer.stem(t))
        return text_result

In [ ]:
sample_text = "Troubling"
sample_text_result = cleanText(sample_text, lemmatize=False, stemmer=True)
sample_text_result = " ".join(str(x) for x in sample_text_result)
print(sample_text)
print(sample_text_result)
sample_text_result = cleanText(sample_text, lemmatize=True, stemmer=False)
sample_text_result = " ".join(str(x) for x in sample_text_result)
print(sample_text_result)

## Section 1.2: BAG OF WORDS

A bag-of-words model, or BoW for short, is a way of extracting features from text for use in modeling, such as with machine learning algorithms.

The approach is very simple and flexible, and can be used in many ways for extracting features from documents.

A bag-of-words is a representation of text that describes the occurrence of words within a document.
It is called a “bag” of words, because any information about the order or structure of words in the document is discarded. The model is only concerned with whether known words occur in the document, not where in the document.

In [ ]:
# Functions to convert document(s) to a list of words, with the option of removing stopwords. Returns document-term matrix.

def createBagOfWords(train, test, remove_stopwords, lemmatize, stemmer):
    if remove_stopwords:
        vectorizer = CountVectorizer(analyzer='word', input='content', stop_words=stopwords.words('english'))
    else:
        vectorizer = CountVectorizer(analyzer='word', input='content')

    clean_train = []
    for paragraph in train:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_train.append(paragraph)

    clean_test = []
    for paragraph in test:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_test.append(paragraph)

    bag_of_words_train = vectorizer.fit_transform(clean_train).toarray()
    bag_of_words_test = vectorizer.transform(clean_test).toarray()
    return bag_of_words_train, bag_of_words_test


## Section 1.3: TF-IDF
TF-IDF technique is used to find meaning of sentences consisting of words and cancels out the incapabilities of Bag of Words technique which is good for text classification or for helping a machine read words in numbers.

The number of times a term occurs in a document is called its Term frequency (TF).

 Document frequency is the number of documents in which the word is present.  Inverse DF (IDF) is the inverse of the document frequency which measures the informativeness of term *t*.




In [ ]:
def createTFIDF(train, test, remove_stopwords, lemmatize, stemmer):
    if remove_stopwords:
        vectorizer = TfidfVectorizer(analyzer='word', input='content', stop_words=stopwords.words('english'))
    else:
        vectorizer =  TfidfVectorizer(analyzer='word', input='content')

    clean_train = []
    for paragraph in train:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_train.append(paragraph)

    clean_test = []
    for paragraph in test:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_test.append(paragraph)

    tfidf_train = vectorizer.fit_transform(clean_train).toarray()
    tfidf_test = vectorizer.transform(clean_test).toarray()
    return tfidf_train, tfidf_test

# **Section 2: UNDERSTANDING THE DATA : A REVIEWS DATASET**

Sentiment analysis is the interpretation and classification of emotions (such as positive, negative and neutral) within text data using text analysis techniques.  
Given below is a dataset consisting of reviews along with sentiment class (positive or negative).

In [ ]:
# Upload the Reviews CSV file that has been shared with you.
# Run this cell, click on the 'Choose files' button and upload the file.
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd
df = pd.read_csv('reviews.csv')

In [ ]:
df = df.dropna()

In [ ]:
df.to_csv('reviews.csv', index=False)

# **Section 3: KNN MODEL**

Given below are two KNN models; in the first case we are using Bag-of-Words and in the second case we are using TF-IDF.
Note the different metrics and parameters used in each.

In [ ]:
from sklearn import metrics, neighbors
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

## TASK - 1: Tweak the models below and see results with different parameters and distance metrics.

def bow_knn():
    """Method for determining nearest neighbors using bag-of-words and K-Nearest Neighbor algorithm"""

    training_data = pd.read_csv('reviews.csv')
    X_train, X_test, y_train, y_test = train_test_split(training_data["sentence"], training_data["sentiment"], test_size=0.2, random_state=5)
    X_train, X_test = createBagOfWords(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    # print(X_train)
    knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='euclidean', metric_params=None, n_jobs=1)

    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print('KNN with BOW accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)
    print('\n')
    return predicted, y_test


def tfidf_knn():
    """Method for determining nearest neighbors using tf-idf and K-Nearest Neighbor algorithm"""

    training_data = pd.read_csv('reviews.csv')
    X_train, X_test, y_train, y_test = train_test_split(training_data["sentence"], training_data["sentiment"],
                                                        test_size=0.2, random_state=5)
    X_train, X_test = createTFIDF(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    # print(X_train)
    knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='distance', algorithm='brute', leaf_size=30, p=2,
                                         metric='cosine', metric_params=None, n_jobs=1)

    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print('KNN with TFIDF accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)
    return predicted, y_test

Note: Cross-validation will be discussed in detail in the upcoming lab session.

In [ ]:
## KNN accuracy after using BoW
predicted, y_test = bow_knn()

In [ ]:
## KNN accuracy after using TFIDF
predicted, y_test = tfidf_knn()

# Section 4: SPAM TEXT DATASET
Now let's use what we've learnt to classify texts as spam or not spam.

In [ ]:
# Upload the spam text data CSV file that has been shared with you. You can also download the file from https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset
# Run this cell, click on the 'Choose files' button and upload the file.
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd
df = pd.read_csv('spam.csv', error_bad_lines=False)
df

In [ ]:
df['Category'] = df['Category'].map({'ham': 0, 'spam': 1})

In [ ]:
df.head(5)

In [ ]:
len(df)

In [ ]:
from sklearn import metrics, neighbors
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

## TASK - 2: Tweak the models below and see results with different parameters and distance metrics.

def bow_knn():
    """Method for determining nearest neighbors using bag-of-words and K-Nearest Neighbor algorithm"""

    training_data = pd.read_csv('spam.csv')
    training_data['Category'] = training_data['Category'].map({'ham': 0, 'spam': 1})
    X_train, X_test, y_train, y_test = train_test_split(training_data["Message"], training_data["Category"], test_size=0.2, random_state=5)
    X_train, X_test = createBagOfWords(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='euclidean', metric_params=None, n_jobs=1)

    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print('KNN with BOW accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)
    print('\n')
    return predicted, y_test


def tfidf_knn():
    """Method for determining nearest neighbors using tf-idf and K-Nearest Neighbor algorithm"""

    training_data = pd.read_csv('spam.csv')
    training_data['Category'] = training_data['Category'].map({'ham': 0, 'spam': 1})
    X_train, X_test, y_train, y_test = train_test_split(training_data["Message"], training_data["Category"], test_size=0.2, random_state=5)
    X_train, X_test = createTFIDF(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='distance', algorithm='brute', leaf_size=30, p=2, metric='cosine', metric_params=None, n_jobs=1)

    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print('KNN with TFIDF accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)
    return predicted, y_test

In [ ]:
# This cell may take some time to run
predicted, y_test = bow_knn()

In [ ]:
# This cell may take some time to run
predicted, y_test = tfidf_knn()

### Questions to Think About and Answer
1. Why does the TF-IDF approach generally result in a better accuracy than Bag-of-Words ?
2. Can you think of techniques that are better than both BoW and TF-IDF ?
3. Read about Stemming and Lemmatization from the resources given below. Think about the pros/cons of each.

1.QUESTION: The TF-IDF (Term Frequency-Inverse Document Frequency) approach is often considered more effective than the Bag-of-Words (BoW) model in certain natural language processing (NLP) tasks because it addresses some limitations of BoW and takes into account the importance of words in a document within a larger corpus. Here are some reasons why TF-IDF may lead to better accuracy compared to BoW:

Term Importance:

TF (Term Frequency): TF measures how often a term appears in a document. However, it doesn't consider the importance of a term in the overall corpus. TF-IDF includes the inverse document frequency (IDF) component, which penalizes terms that are too common across all documents and boosts the importance of terms that are specific to a document.
Document Relevance:

IDF (Inverse Document Frequency): This component in TF-IDF helps to identify words that are important to a specific document by downplaying the significance of words that are common across all documents. Words that appear frequently across many documents (like "the," "and," etc.) are given lower importance.
Normalization:

Normalization of TF: TF-IDF normalizes the term frequencies, preventing longer documents from having higher overall term frequencies. This normalization helps to give equal importance to terms regardless of the document length.
Sparse Representation:

Sparse Representation: TF-IDF representation tends to be sparse, meaning that the majority of elements in the representation are zero. This can be beneficial for computational efficiency and memory usage. BoW, on the other hand, can result in dense representations, especially in large datasets.
Contextual Information:

Consideration of Word Context: TF-IDF considers the context of words in the document by assigning weights based on the importance of words within the document and across the corpus. BoW, in its simplest form, treats each word independently and doesn't capture the relationships between words.
Better Handling of Stop Words:

Handling of Stop Words: TF-IDF tends to handle stop words (common words like "and," "the," etc.) better than BoW. Since TF-IDF downplays the importance of common words, they are less likely to influence the representation significantly.
While TF-IDF has its advantages, it's worth noting that the choice between TF-IDF and BoW depends on the specific task and the characteristics of the dataset. In some cases, simpler models like BoW may be sufficient, while in others, more sophisticated methods such as word embeddings or deep learning approaches might be preferred.

2.QUESTION: Certainly! While Bag-of-Words (BoW) and TF-IDF are common and effective text representation techniques, there are more advanced methods that often perform better in capturing semantic relationships and context in natural language. Some of these techniques include:

Word Embeddings:

Techniques like Word2Vec, GloVe, and FastText create dense vector representations for words, capturing semantic relationships and context. These embeddings are pre-trained on large corpora and can be fine-tuned for specific tasks.
Doc2Vec (Paragraph Vectors):

An extension of Word2Vec, Doc2Vec learns fixed-size vector representations for entire documents. This approach considers the context of words in the document and captures document-level semantics.
BERT (Bidirectional Encoder Representations from Transformers):

BERT, and similar transformer-based models, use attention mechanisms to capture contextual information bidirectionally. BERT embeddings have shown state-of-the-art performance in various NLP tasks and can be fine-tuned for specific applications.
ELMo (Embeddings from Language Models):

ELMo generates contextualized word embeddings by considering the context in which a word appears in a sentence. It captures word meanings that may vary based on their context.
ULMFiT (Universal Language Model Fine-tuning):

ULMFiT is a transfer learning approach where a language model is pre-trained on a large dataset and fine-tuned for a specific task. It has been successful in various NLP applications.
Attention Mechanisms:

Attention mechanisms, as seen in transformer models, allow the model to focus on different parts of the input sequence when making predictions. This helps capture long-range dependencies and improves performance in tasks like machine translation and sentiment analysis.
Character-level Embeddings:

Representing words at the character level can be beneficial for handling out-of-vocabulary words and morphologically rich languages. Character embeddings can be combined with word embeddings for a more robust representation.
Ensemble Models:

Combining multiple models, each trained with different techniques or features, can often lead to improved performance. Ensemble models can help mitigate the weaknesses of individual models.
Subword Embeddings:

Methods like Byte Pair Encoding (BPE) and SentencePiece can be used to represent words as a combination of subword units, allowing the model to handle rare or out-of-vocabulary words more effectively.
The choice of technique depends on the specific task, dataset characteristics, and available resources. State-of-the-art models in NLP often leverage a combination of these techniques and incorporate them into larger architectures to achieve superior performance on various tasks.

3.QUESTION: I don't have the ability to browse external resources in real-time. However, I can provide you with a general overview of stemming and lemmatization, along with their pros and cons based on my training data up to January 2022.

Stemming:
Definition:
Stemming is the process of reducing words to their base or root form. It involves removing suffixes or prefixes to obtain a common base form of words.

Pros:

Computational Efficiency: Stemming is computationally less expensive compared to lemmatization, making it faster.
Simplicity: Stemming is a simpler process, and it's easier to implement.
Cons:

Over-Stemming: Stemming can sometimes result in overly aggressive word reductions, leading to the loss of meaning. For example, "running" and "runner" might both be reduced to "run."
Lack of Word Variants: Stemming may not capture all possible word variants, as it uses heuristics to truncate words.
Lemmatization:
Definition:
Lemmatization is the process of reducing words to their base or dictionary form, called the lemma. It involves considering the context and meaning of a word.

Pros:

Preservation of Meaning: Lemmatization retains the core meaning of words, providing more accurate representations.
Improved Accuracy: Lemmatization is less prone to over-stemming compared to stemming, as it considers the linguistic context.
Cons:

Computational Complexity: Lemmatization is computationally more expensive than stemming, requiring access to lexical resources.
Complex Implementation: Implementing lemmatization often involves the use of linguistic resources such as dictionaries or databases, making it more complex than stemming.
Comparison:
Use Case: Stemming is often preferred in information retrieval or search engine applications where computational efficiency is critical. Lemmatization is more suitable for tasks where maintaining the accuracy of word meanings is crucial, such as in natural language understanding or sentiment analysis.

Precision vs. Speed: Stemming sacrifices some precision for speed, while lemmatization aims to provide more accurate results at the cost of computational complexity.

Language Specificity: Stemming algorithms may be language-specific, while lemmatization often requires language-specific dictionaries or resources.

In summary, the choice between stemming and lemmatization depends on the specific requirements of the NLP task, the desired trade-off between computational efficiency and linguistic accuracy, and the characteristics of the dataset. Many NLP applications may opt for lemmatization when precision and context preservation are critical, while stemming may be preferred in scenarios where speed and simplicity are prioritized.

### Useful Resources for further reading
1. Stemming and Lemmatization: https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html
2. TF-IDF and BoW : https://www.analyticsvidhya.com/blog/2020/02/quick-introduction-bag-of-words-bow-tf-idf/
3. TF-IDF: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html
